In [1]:
import os
localfs_path = os.environ.get('SCRATCH_LOCAL') + '/'

In [2]:
os.environ['_JAVA_OPTIONS'] = f'-Djava.io.tmpdir={localfs_path}'

import hail as hl

hl.init(
    tmp_dir=(localfs_path+'tmp_hail_3'),
    spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'},
    default_reference='GRCh38'
) 

Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4130551/
Picked up _JAVA_OPTIONS: -Djava.io.tmpdir=/localfs/4130551/


23/07/27 08:12:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.3.2
SparkUI available at http://ac0422:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.113-cf32652c5077
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/preprocessing/oligogenic-model/hail-20230727-0812-0.2.113-cf32652c5077.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
import numpy as np
hl.plot.output_notebook()
import bokeh
from scipy import stats

from bokeh.plotting import figure, output_notebook, show
output_notebook

Loading BokehJS ...

<function bokeh.io.output.output_notebook(resources=None, verbose=False, hide_banner=False, load_timeout=5000, notebook_type='jupyter')>

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from oligogenic_model import (
    read_in_pheno_anno_all
)

In [ ]:
mt = read_in_pheno_anno_all(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt',
    'all')

2023-07-26 21:08:21.501 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)
  Loading field 'f4' as type str (not specified)
  Loading field 'f5' as type str (not specified)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type str (not specified)
  Loading field 'f8' as type str (not specified)
2023-07-26 21:08:22.513 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)
2023-07-26 21:08:23.807 Hail: INFO: Reading table to impute column types
2023-07-26 21:08:24.820 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading

In [11]:
mt = hl.read_matrix_table('/localfs/4130551/all.mt')

In [12]:
samples_to_exclude = ['S_7288', 'S_7289', 'S_7290', 'WGS_6827', 'WGS_6835', 'B502', 'B506']

In [13]:
mt = mt.filter_cols(
    hl.literal(samples_to_exclude).contains(mt.s),
    keep = False
)

In [14]:
mt.count()

(6538550, 280)

In [15]:
mt.aggregate_cols(hl.agg.counter(mt.group))

{'GTS': 180, 'local_controls': 100}

In [16]:
gts = mt.filter_cols(mt.group == 'GTS')
sport = mt.filter_cols(mt.group == 'local_controls')

In [17]:
mt = mt.filter_rows(hl.agg.any(mt.GT.is_non_ref()))

In [18]:
mt = mt.naive_coalesce(500)

In [19]:
gts = gts.filter_rows(hl.agg.any(gts.GT.is_non_ref()))

In [20]:
gts = gts.naive_coalesce(500)

In [21]:
sport = sport.filter_rows(hl.agg.any(sport.GT.is_non_ref()))

In [22]:
sport = sport.naive_coalesce(500)

In [23]:
mt.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt',
    overwrite = True
)

2023-07-27 08:21:20.712 Hail: INFO: wrote matrix table with 6514756 rows and 280 columns in 488 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt


In [24]:
gts.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-gts.mt',
    overwrite = True
)

sport.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-sport.mt',
    overwrite = True
)

2023-07-27 08:24:45.589 Hail: INFO: wrote matrix table with 5209343 rows and 180 columns in 488 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-gts.mt
2023-07-27 08:27:35.852 Hail: INFO: wrote matrix table with 4452374 rows and 100 columns in 488 partitions to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-sport.mt


In [25]:
mt = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-all.mt')

In [26]:
gts = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-gts.mt')

In [27]:
sport = hl.read_matrix_table('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/for-qc-sport.mt')

In [28]:
mt.count()

(6514756, 280)

In [36]:
mt.aggregate_rows(
    hl.agg.count_where(
        hl.is_snp(
            mt.alleles[0],
            mt.alleles[1]
        )
    )
)

6514756

In [29]:
gts.count()

(5209343, 180)

In [37]:
gts.aggregate_rows(
    hl.agg.count_where(
        hl.is_snp(
            gts.alleles[0],
            gts.alleles[1]
        )
    )
)

5209343

In [30]:
sport.count()

(4452374, 100)

In [38]:
sport.aggregate_rows(
    hl.agg.count_where(
        hl.is_snp(
            sport.alleles[0],
            sport.alleles[1]
        )
    )
)

4452374

In [23]:
gts1 = gts.filter_cols(gts.phenotypes.family == '.')

In [29]:
gts1.count()

(6538550, 39)

In [35]:
gts1.aggregate_cols(
    hl.agg.counter(
        gts1.phenotypes.add_pheno
    )
)

{'.': 7,
 'ADHD': 2,
 'ADHDANX': 1,
 'ADHDMDDANX': 4,
 'ADHDOCD': 1,
 'ADHDOCDANX': 3,
 'ADHDOCDMDD': 1,
 'ADHDOCDMDDANX': 3,
 'ANX': 3,
 'ASD': 1,
 'ASDADHD': 1,
 'ASDADHDANX': 1,
 'ASDMDDANX': 1,
 'MDD': 1,
 'OCD': 2,
 'OCDMDD': 2,
 'OCDMDDANX': 4,
 'SCHIMDDANX': 1}

In [34]:
gts1.aggregate_cols(
    hl.agg.counter(
        gts1.phenotypes.sex
    )
)

{'F': 6, 'M': 33}

In [28]:
gts2 = gts.filter_cols(gts.phenotypes.family == '.', keep = False)

In [42]:
gts2.count()

(6538550, 141)

In [44]:
gts2.aggregate_cols(
    hl.agg.counter(
        gts2.phenotypes.family
    )
)

{'A': 7,
 'B': 7,
 'C': 7,
 'D': 7,
 'E': 7,
 'F': 11,
 'G': 4,
 'H': 6,
 'I': 6,
 'J': 5,
 'K': 3,
 'L': 3,
 'M': 3,
 'N': 3,
 'O': 3,
 'P': 4,
 'R': 5,
 'S': 6,
 'T': 9,
 'U': 5,
 'W': 15,
 'X': 5,
 'Y': 8,
 'Z': 2}

In [36]:
gts2.aggregate_cols(
    hl.agg.counter(
        gts2.phenotypes.heavy_tics_familial
    )
)

{'.': 138, 'YES': 3}

In [39]:
gts2.aggregate_cols(
    hl.agg.counter(
        gts2.phenotypes.phenotype
    )
)

{'.': 55, 'GTS': 46, 'tics': 40}

In [40]:
gts2.aggregate_cols(
    hl.agg.group_by(
        gts2.phenotypes.phenotype,
        hl.agg.counter(
            gts2.phenotypes.add_phen
        )
    )
)

{'.': {'.': 53, 'ASD': 1, 'MDD': 1},
 'GTS': {'.': 28,
  'ADHD': 1,
  'ADHDANX': 1,
  'ADHDOCDMDDANX': 1,
  'ANX': 5,
  'ASD': 5,
  'OCDMDD': 2,
  'OCDMDDANX': 3},
 'tics': {'.': 35, 'ASD': 1, 'OCD': 4}}

In [ ]:
gts1.aggregate_cols

In [16]:
mt.aggregate_cols(hl.agg.counter(mt.group))

{'GTS': 180, 'local_controls': 102}

In [18]:
samples = mt.cols()
samples = samples.select()

In [41]:
samples.describe()

NameError: name 'samples' is not defined

In [24]:
samples.export('samples_for_prs.tsv')

2023-07-14 12:55:09.286 Hail: INFO: Coerced sorted dataset
2023-07-14 12:55:09.611 Hail: INFO: merging 17 files totalling 1.8K...
2023-07-14 12:55:09.624 Hail: INFO: while writing:
    samples_for_prs.tsv
  merge time: 12.570ms


### len of selected gene lists

In [14]:
gene_list = ['DCC', 'RBFOX', 'SLC30A9', 'DCAF4L1', 'SORCS3', 'KCNQ5', 'KCNQ-IT1', 'APOPT1', 'C14orf2', 'NAA11', 'NEGR1',
        'CHADL', 'SOX5', 'PANK2', 'COL27A1', 'PDGFB', 'CELSR3', 'OPA1', 'FBN2', 'WWC1', 'NIPBL', 
             'FN1', 'FBN2', 'SLITRK1', 'SLITRK2', 'SLITRK3', 'SLITRK4', 'SLITRK5', 'SLITRK6', 
             'HDC', 'OPRK1', 'PCDH10', 'NTSR2', 'CHD8', 'SCUBE1', 'PNKD', 'CNTNAP2', 'MOG', 
             'DRD2', 'DRD3', 'DRD4', 'DRD5', 'DAT1', 'DBH', 'HTR2A', 'TPH2', 'EAAT1', 'SAPAP3',
            'CTNNA3', 'NLGN4', 'FSCB', 'IMMP2L', 'NRXN1', 'AADAC', 'DBH', 'MAOA', 'HTR1A', 'HTR2C', 'SLC6A4',
             'TPH2', 'COL27A1', '5-HTTLPR', 'EAAT1', 'COL8A1', 'KCNE1', 'KCNE2',
         'RICTOR', 'WWC1', 'CELSR3, NIPBL', 'FN1', 'PNKD', 'CDH26', 'CADM2', 'OPCML', 'CDH9',
         'NCAM2', 'CD47', 'CDH5', 'CADM4', 'C1QBP', 'CTTN', 'LSAMP',
         'PKP4', 'PCDH1', 'CNTNAP2', 'MBP', 'GABBR2', 'GABBR2', 'GRIK4', 'NCR1', 'FLT3', 'IL12A', 'HDAC9',
         'CD180', 'CDH26', 'NCAM2', 'NTM', 'ROBO2', 'NR2F1', 'ASH1L', 'FLT3']

In [15]:
len(gene_list)

101

### how many variants in the model?

In [18]:
mt = read_in_pheno_anno_sport(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/gts_and_s.mt',
    'gts_sport_for_vars'
)

2023-07-10 18:08:30.474 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type str (not specified)
  Loading field 'f4' as type str (not specified)
  Loading field 'f5' as type str (not specified)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type str (not specified)
  Loading field 'f8' as type str (not specified)
2023-07-10 18:08:30.981 Hail: INFO: Reading table without type imputation
  Loading field 'Gene stable ID' as type str (not specified)
  Loading field 'UniProtKB Gene Name symbol' as type str (not specified)
2023-07-10 18:08:31.864 Hail: INFO: Reading table to impute column types
2023-07-10 18:08:32.365 Hail: INFO: Finished type imputation
  Loading field 'ID' as type str (imputed)
  Loading field 'family' as type str (imputed)
  Loading field 'sex' as type str (imputed)
  Loading

In [7]:
mt = hl.read_matrix_table(localfs_path+'gts_sport_for_vars.mt')

In [8]:
genes = ['CHADL', 'DRD2', 'MAOA', 'PCDH10', 'HTR2A', 'SLITRK5', 'SORCS3', 'KCNQ5', 'CDH9', 'CHD8']

In [9]:
mt = mt.filter_rows(
    hl.literal(genes).contains(mt.nearest_genes_20kb)
)

In [13]:
mt = mt.filter_rows(mt.cadd.score_phred > 0)

In [14]:
mt.count()

(7654, 204)

In [16]:
mt.aggregate_rows(hl.agg.stats(mt.gnomad_v_3_1.freq.AF[2]))

Struct(mean=0.16338312177098835, stdev=0.26313946535076954, min=0.0, max=1.0, n=5895, sum=963.1435028399763)

In [19]:
mt.aggregate_rows(hl.agg.approx_median(mt.gnomad_v_3_1.freq.AF[2]))

0.01998941736727615